In [2]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
import gradio as gr
import io
from PIL import Image

# Train the AI Model
def train_model(data):
    encoder = LabelEncoder()
    data['Category'] = encoder.fit_transform(data['Category'])

    # Preprocess text using TF-IDF
    vectorizer = TfidfVectorizer(stop_words='english')
    X_text = vectorizer.fit_transform(data['Description'])

    # Combine TF-IDF with Amount
    X_amount = data['Amount'].values.reshape(-1, 1)
    X = np.hstack([X_text.toarray(), X_amount])

    # Target variable
    y = data['Category']

    # Train SVM model
    model = SVC(kernel='linear', random_state=42)
    model.fit(X, y)

    return model, vectorizer, encoder

# Generate Suggestions
def generate_suggestions(category_summary, total_spent):
    suggestions = []
    for category, amount in category_summary.items():
        percentage = (amount / total_spent) * 100
        if percentage > 30:
            suggestions.append(f"You might be spending too much on {category} ({percentage:.1f}%). Consider ways to reduce it.")
    return suggestions

# Process Expenses and Predict Categories
def process_expenses(file, model, vectorizer, encoder):
    try:
        # Read the uploaded CSV file
        data = pd.read_csv(file)

        if not all(col in data.columns for col in ['Description', 'Amount']):
            return "Error: CSV must contain 'Description' and 'Amount' columns.", None, None

        # Predict categories if not provided
        if 'Category' not in data.columns:
            text_features = vectorizer.transform(data['Description'])
            amount_features = data['Amount'].values.reshape(-1, 1)
            combined_features = np.hstack([text_features.toarray(), amount_features])
            data['Category'] = encoder.inverse_transform(model.predict(combined_features))

        # Summarize expenses
        summary = data.groupby('Category')['Amount'].sum()
        total_expenses = summary.sum()
        suggestions = generate_suggestions(summary, total_expenses)

        # Create visualizations
        plt.figure(figsize=(10, 6))
        sns.barplot(x=summary.index, y=summary.values)
        plt.title('Monthly Expense Distribution')
        plt.xlabel('Category')
        plt.ylabel('Amount ($)')
        bar_graph_io = io.BytesIO()
        plt.savefig(bar_graph_io, format='png')
        bar_graph_io.seek(0)
        bar_graph_image = Image.open(bar_graph_io)
        plt.close()

        plt.figure(figsize=(8, 8))
        plt.pie(summary.values, labels=summary.index, autopct='%1.1f%%', startangle=90)
        plt.title('Monthly Expense Breakdown')
        pie_chart_io = io.BytesIO()
        plt.savefig(pie_chart_io, format='png')
        pie_chart_io.seek(0)
        pie_chart_image = Image.open(pie_chart_io)
        plt.close()

        summary_text = "\n".join([f"{cat}: {amount}$" for cat, amount in summary.items()])
        suggestion_text = "Suggestions:\n" + "\n".join(suggestions)

        output = f"{summary_text}\n\n{suggestion_text}"

        return output, bar_graph_image, pie_chart_image

    except Exception as e:
        return f"Error processing file: {e}", None, None

# Interface
def setup_interface():

    model, vectorizer, encoder = train_model(pd.read_csv('replace here expenses_.csv file in github'))

    # Gradio interface
    interface = gr.Interface(
        fn=lambda file: process_expenses(file, model, vectorizer, encoder),
        inputs=gr.File(label="Upload CSV File"),
        outputs=[
            gr.Textbox(label="Expense Summary and Suggestions"),
            gr.Image(label="Expense Distribution Bar Chart"),
            gr.Image(label="Expense Breakdown Pie Chart")
        ],
        title="AI-Powered Expense Tracker",
        description="Upload a CSV file with 'Description' and 'Amount' columns. Optional: Include 'Category' for better insights."
    )
    return interface

interface = setup_interface()
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d0e8afa05694633ed0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
